In [1]:
import pandas as pd
import numpy as np
#from feature_generators import TS_feature_generator_functions as fg
from timeit import default_timer as dt
from datetime import datetime
import re
from sklearn.externals import joblib
from time import time
from collections import Counter

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
pd.set_option('display.max_columns', None)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
data_dir = '/home/flowcast/data/python/data_validate_180315/'

In [5]:
ls /home/flowcast/data/python/data_validate_180315/

all_prod_180319.csv           data_3NF_180316_IFNS.csv
all_prod_validate_180319.csv  data_3NF_180316_LBDS.csv
buyer_supplier_3m.csv         data_3NF_180316_LRRP.csv
buyer_supplier_6m.csv         data_3NF_180316_PSHP.csv
buyer_supplier_all.csv        data_3NF_180316_RSRV.csv
buyer_supplier_prevyr.csv     data_3NF_180316_TRDS.csv
cast_columns_template.csv     data_CG.csv
ccy_features_vector.csv       data_for_model.csv
ccy_features_vol.csv          data_for_model.zip
client_3m.csv                 dr_logreg_scores_180223.csv
client_6m.csv                 dr_rf_preds.csv
client_all.csv                LD_logreg_scores_180223.csv
client_prevyr.csv             ld_rf_preds.csv
data_3NF_180316_BATB.csv      lr_dr_180223.joblib
data_3NF_180316_BDBR.csv      lr_ld_180223.joblib
data_3NF_180316_ECIF.csv      minmaxscaler_180223.pkl
data_3NF_180316_ECPR.csv      rf_dr_180223.joblib
data_3NF_180316_IFNB.csv      rf_ld_180223.joblib


In [6]:
data = pd.read_csv(data_dir+'data_for_model.csv')
data.head()


/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cty_code,doc_id,supplier_id,buyer_id,doc_type_code,deal_id,fin_flag,rec_id,doc_date,doc_ccy_code,doc_ccy_amt,tenor,tenor_start_date,due_date,grace_period,maturity_date,fin_tenor_start_date,fin_tenor,to_seq_no,from_seq_no,pymt_ccy_code,pymt_alloc_amt,fin_ccy_code,fin_ccy_cp_alloc_amt,fin_ccy_ncp_alloc_amt,fin_ccy_ncp_os_amt,fin_ccy_cp_os_amt,fin_ccy_amt,os_ccy_amt,fin_date,sale_ledger_flag,pool_based_flag,step_code,product_variant_code,rel_seq_no,pymt_date,settled_flag,late_days,finance_days,finance_due_days,isLate,late_days_bucket,fin_xrate,pymt_xrate,doc_xrate,doc_usd_amt,fin_usd_amt,os_usd_amt,pymt_usd_amt,dr,dr_bucket,product_,client_id,le_id,lid_isic_cd_val,industry_code,CG,buyer_supplier_yrs,buyer_supplier_past_inv_cnt,buyer_supplier_annual_inv_cnt,buyer_supplier_ave_fin_amt,buyer_supplier_ave_annual_fin_amt,buyer_supplier_std_fin_amt,buyer_supplier_late_days,buyer_supplier_isLate,buyer_supplier_dr,buyer_supplier_term,buyer_supplier_cty,buyer_supplier_prevyr_inv_cnt,buyer_supplier_prevyr_ave_fin_amt,buyer_supplier_prevyr_std_fin_amt,buyer_supplier_prevyr_late_days,buyer_supplier_prevyr_isLate,buyer_supplier_prevyr_dr,buyer_supplier_prevyr_term,buyer_supplier_season_x,buyer_supplier_6m_inv_cnt,buyer_supplier_6m_ave_fin_amt,buyer_supplier_6m_std_fin_amt,buyer_supplier_6m_dr,buyer_supplier_6m_term,buyer_supplier_6m_late_days,buyer_supplier_6m_isLate,buyer_supplier_3m_inv_cnt,buyer_supplier_3m_ave_fin_amt,buyer_supplier_3m_std_fin_amt,buyer_supplier_3m_dr,buyer_supplier_3m_term,buyer_supplier_3m_late_days,buyer_supplier_3m_isLate,client_yrs,client_past_inv_cnt,client_annual_inv_cnt,client_ave_fin_amt,client_ave_annual_fin_amt,client_std_fin_amt,client_late_days,client_isLate,client_dr,client_term,client_cty,client_3m_inv_cnt,client_3m_ave_fin_amt,client_3m_std_fin_amt,client_3m_dr,client_3m_term,client_3m_late_days,client_3m_isLate,client_season_x,client_6m_inv_cnt,client_6m_ave_fin_amt,client_6m_std_fin_amt,client_6m_dr,client_6m_term,client_6m_late_days,client_6m_isLate,client_prevyr_inv_cnt,client_prevyr_ave_fin_amt,client_prevyr_std_fin_amt,client_prevyr_late_days,client_prevyr_isLate,client_prevyr_dr,client_prevyr_term,first_client_invoice,RICI_index,due_date_dow,maturity_date_dow,buyer_centric_flag,ccy_wk_vector,ccy_3m_vector,ccy_6m_vector,ccy_yr_vector,ccy_tenor_vector,ccy_fintenor_vector,ccy_wk_vol,ccy_3m_vol,ccy_6m_vol,ccy_yr_vol,ccy_tenor_vol,ccy_fintenor_vol
0,AE,RFHLM9WX4V-89034,100369293,100244505.0,INV,TF123BF562,Y,6728531788,2017-03-24,USD,113268.50,45,2017-03-24 00:00:00,2017-05-08,0,2017-05-08,2017-03-25 00:00:00,44.0,37.0,1,USD,113268.50,USD,0.0,113268.50,0.0,0.0,113268.50,0.0,2017-03-25,Y,N,SETT,IFNB02,37.0,2017-05-04,Y,0,40,44,0,0,1.000000,1.000000,1.000000,113268.500000,113268.500000,0.0,113268.500000,0.0,0,IFNB,100244505,11545630.0,6116.0,61,5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2229.32,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AE,V5HKSF85PW-36933,100369293,100244505.0,INV,TF123BF144,Y,783293897,2017-03-10,USD,106717.15,45,2017-03-10 00:00:00,2017-04-24,0,2017-04-24,2017-03-11 00:00:00,44.0,29.0,1,USD,106717.15,USD,0.0,106717.15,0.0,0.0,106717.15,0.0,2017-03-11,Y,N,SETT,IFNB02,29.0,2017-04-09,Y,0,29,44,0,0,1.000000,1.000000,1.000000,106717.150000,106717.150000,0.0,106717.150000,0.0,0,IFNB,100244505,11545630.0,6116.0,61,5A,1.0,7.0,7.0,107842.878571,754900.150000,43236.173824,0.0,0.0,0.000000,44.428571,1.0,7.0,107842.878571,43236.173824,0.0,0.0,0.000000,44.428571,0.989561,7.0,107842.878571,43236.173824,0.000000,44.428571,0.0,0.0,7.0,107842.878571,43236.173824,0.000000,44.428571,0.0,0.0,1.0,7.0,7.0,107842.878571,754900.150000,43236.173824,0.000000,0.0,0.000000,44.428571,1.0,7.0,107842.878571,43236.173824,0.000000,44.428571,0.000000,0.0,0.989561,7.0,107842.878571,432

In [7]:
#check for columns with missing values:

na_cols = data.columns[data.isnull().any()]
na_cols

Index([u'fin_tenor', u'to_seq_no', u'rel_seq_no', u'le_id', u'lid_isic_cd_val',
       u'buyer_supplier_yrs', u'buyer_supplier_past_inv_cnt',
       u'buyer_supplier_annual_inv_cnt', u'buyer_supplier_ave_fin_amt',
       u'buyer_supplier_ave_annual_fin_amt', u'buyer_supplier_std_fin_amt',
       u'buyer_supplier_late_days', u'buyer_supplier_isLate',
       u'buyer_supplier_dr', u'buyer_supplier_term', u'buyer_supplier_cty',
       u'buyer_supplier_prevyr_inv_cnt', u'buyer_supplier_prevyr_ave_fin_amt',
       u'buyer_supplier_prevyr_std_fin_amt',
       u'buyer_supplier_prevyr_late_days', u'buyer_supplier_prevyr_isLate',
       u'buyer_supplier_prevyr_dr', u'buyer_supplier_prevyr_term',
       u'buyer_supplier_season_x', u'buyer_supplier_6m_inv_cnt',
       u'buyer_supplier_6m_ave_fin_amt', u'buyer_supplier_6m_std_fin_amt',
       u'buyer_supplier_6m_dr', u'buyer_supplier_6m_term',
       u'buyer_supplier_6m_late_days', u'buyer_supplier_6m_isLate',
       u'buyer_supplier_3m_inv_cnt', u

In [8]:
data_orig = data

In [9]:
print(len(data))
data['CG'] = np.where(data.CG=='no_CG','NO_CG', data.CG)
data = data[data.cty_code!='ZM']
print(len(data))

296621
296620


In [13]:
data['late_days_bucket'] = np.where(data.late_days>60, 3, np.where(data.late_days>30, 2,np.where(data.late_days>0,1,0)))

/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
data.to_csv(data_dir+'data_for_model.csv', index= False)

In [24]:
# change missing data:

for i in range(len(na_cols)):
    col = na_cols[i]
    data[col] = np.where(pd.isnull(data[col]),0,data[col])

data.describe()

/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,supplier_id,buyer_id,doc_ccy_amt,tenor,grace_period,fin_tenor,to_seq_no,from_seq_no,pymt_alloc_amt,fin_ccy_cp_alloc_amt,fin_ccy_ncp_alloc_amt,fin_ccy_ncp_os_amt,fin_ccy_cp_os_amt,fin_ccy_amt,os_ccy_amt,rel_seq_no,late_days,finance_days,finance_due_days,isLate,late_days_bucket,fin_xrate,pymt_xrate,doc_xrate,doc_usd_amt,fin_usd_amt,os_usd_amt,pymt_usd_amt,dr,dr_bucket,client_id,le_id,lid_isic_cd_val,industry_code,buyer_supplier_yrs,buyer_supplier_past_inv_cnt,buyer_supplier_annual_inv_cnt,buyer_supplier_ave_fin_amt,buyer_supplier_ave_annual_fin_amt,buyer_supplier_std_fin_amt,buyer_supplier_late_days,buyer_supplier_isLate,buyer_supplier_dr,buyer_supplier_term,buyer_supplier_cty,buyer_supplier_prevyr_inv_cnt,buyer_supplier_prevyr_ave_fin_amt,buyer_supplier_prevyr_std_fin_amt,buyer_supplier_prevyr_late_days,buyer_supplier_prevyr_isLate,buyer_supplier_prevyr_dr,buyer_supplier_prevyr_term,buyer_supplier_season_x,buyer_supplier_6m_inv_cnt,buyer_supplier_6m_ave_fin_amt,buyer_supplier_6m_std_fin_amt,buyer_supplier_6m_dr,buyer_supplier_6m_term,buyer_supplier_6m_late_days,buyer_supplier_6m_isLate,buyer_supplier_3m_inv_cnt,buyer_supplier_3m_ave_fin_amt,buyer_supplier_3m_std_fin_amt,buyer_supplier_3m_dr,buyer_supplier_3m_term,buyer_supplier_3m_late_days,buyer_supplier_3m_isLate,client_yrs,client_past_inv_cnt,client_annual_inv_cnt,client_ave_fin_amt,client_ave_annual_fin_amt,client_std_fin_amt,client_late_days,client_isLate,client_dr,client_term,client_cty,client_3m_inv_cnt,client_3m_ave_fin_amt,client_3m_std_fin_amt,client_3m_dr,client_3m_term,client_3m_late_days,client_3m_isLate,client_season_x,client_6m_inv_cnt,client_6m_ave_fin_amt,client_6m_std_fin_amt,client_6m_dr,client_6m_term,client_6m_late_days,client_6m_isLate,client_prevyr_inv_cnt,client_prevyr_ave_fin_amt,client_prevyr_std_fin_amt,client_prevyr_late_days,client_prevyr_isLate,client_prevyr_dr,client_prevyr_term,first_client_invoice,RICI_index,due_date_dow,maturity_date_dow,buyer_centric_flag,ccy_wk_vector,ccy_3m_vector,ccy_6m_vector,ccy_yr_vector,ccy_tenor_vector,ccy_fintenor_vector,ccy_wk_vol,ccy_3m_vol,ccy_6m_vol,ccy_yr_vol,ccy_tenor_vol,ccy_fintenor_vol
count,2.966200e+05,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,2.966200e+05,2.966200e+05,296620.000000,296620.000000,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,2.966200e+05,296620.000000,2.966200e+05,296620.000000,296620.000000,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,296620.000000,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,2.966200e+05,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000,296620.000000
mean,1.002386e+08,1.002482e+08,1.075823e+06,57.219628,0.540759,69.722429,6113.013671,5855.261934,1.076429e+06,2.512097e+05,8.243791e+05,1.854492,16.690425,1.075589e+06,18.544916,6113.013671,3.901345,47.780558,44.548877,0.584698,0.611806,0.463951,

In [15]:
for col in data.columns:
    print(col)

cty_code
doc_id
supplier_id
buyer_id
doc_type_code
deal_id
fin_flag
rec_id
doc_date
doc_ccy_code
doc_ccy_amt
tenor
tenor_start_date
due_date
grace_period
maturity_date
fin_tenor_start_date
fin_tenor
to_seq_no
from_seq_no
pymt_ccy_code
pymt_alloc_amt
fin_ccy_code
fin_ccy_cp_alloc_amt
fin_ccy_ncp_alloc_amt
fin_ccy_ncp_os_amt
fin_ccy_cp_os_amt
fin_ccy_amt
os_ccy_amt
fin_date
sale_ledger_flag
pool_based_flag
step_code
product_variant_code
rel_seq_no
pymt_date
settled_flag
late_days
finance_days
finance_due_days
isLate
late_days_bucket
fin_xrate
pymt_xrate
doc_xrate
doc_usd_amt
fin_usd_amt
os_usd_amt
pymt_usd_amt
dr
dr_bucket
product_
client_id
le_id
lid_isic_cd_val
industry_code
CG
buyer_supplier_yrs
buyer_supplier_past_inv_cnt
buyer_supplier_annual_inv_cnt
buyer_supplier_ave_fin_amt
buyer_supplier_ave_annual_fin_amt
buyer_supplier_std_fin_amt
buyer_supplier_late_days
buyer_supplier_isLate
buyer_supplier_dr
buyer_supplier_term
buyer_supplier_cty
buyer_supplier_prevyr_inv_cnt
buyer_supplier

In [16]:
feature_set = data[['cty_code', 
#'doc_id', 
# 'supplier_id', 
# 'buyer_id', 
# 'doc_type_code', 
# 'deal_id', 
# 'fin_flag', 
# 'rec_id', 
# 'doc_date', 
'doc_ccy_code', 
# 'doc_ccy_amt', 
'tenor', 
# 'tenor_start_date', 
# 'due_date', 
'grace_period', 
# 'maturity_date', 
# 'fin_tenor_start_date', 
'fin_tenor', 
# 'to_seq_no', 
# 'from_seq_no', 
'fin_ccy_code', 
# 'fin_ccy_cp_alloc_amt', 
# 'fin_ccy_ncp_alloc_amt', 
# 'fin_ccy_ncp_os_amt', 
# 'fin_ccy_cp_os_amt', 
# 'fin_ccy_amt', 
# 'os_ccy_amt', 
# 'pymt_ccy_amt', 
# 'dr', 
# 'fin_date', 
# 'sale_ledger_flag', 
# 'pool_based_flag', 
# 'step_code', 
'product_variant_code', 
# 'rel_seq_no', 
# 'pymt_date', 
# 'settled_flag', 
# 'late_days', 
# 'finance_days', 
# 'finance_due_days', 
# 'isLate', 
# 'late_days_bucket', 
# 'dr_bucket', 
'doc_usd_amt', 
# 'fin_usd_amt', 
# 'os_usd_amt', 
# 'pymt_usd_amt', 
'product_', 
# 'fin_rate', 
'due_date_dow', 
'maturity_date_dow', 
'buyer_centric_flag', 
'CG', 
'ccy_wk_vol', 
'ccy_3m_vol', 
'ccy_6m_vol', 
'ccy_yr_vol', 
'ccy_tenor_vol', 
'ccy_fintenor_vol', 
'ccy_wk_vector', 
'ccy_3m_vector', 
'ccy_6m_vector', 
'ccy_yr_vector', 
'ccy_tenor_vector', 
'ccy_fintenor_vector', 
'RICI_index', 
# 'test_split', 
'buyer_supplier_prevyr_inv_cnt', 
'buyer_supplier_prevyr_ave_fin_amt', 
'buyer_supplier_prevyr_std_fin_amt', 
'buyer_supplier_prevyr_late_days', 
# 'buyer_supplier_prevyr_isLate', 
'buyer_supplier_prevyr_dr', 
'buyer_supplier_prevyr_term', 
'buyer_supplier_season_x', 
'buyer_supplier_yrs', 
'buyer_supplier_past_inv_cnt', 
'buyer_supplier_annual_inv_cnt', 
'buyer_supplier_ave_fin_amt', 
'buyer_supplier_ave_annual_fin_amt', 
'buyer_supplier_std_fin_amt', 
'buyer_supplier_late_days', 
#'buyer_supplier_isLate', 
# 'buyer_supplier_dr', 
'buyer_supplier_term', 
'buyer_supplier_cty', 
'buyer_supplier_3m_inv_cnt', 
'buyer_supplier_3m_ave_fin_amt', 
'buyer_supplier_3m_std_fin_amt', 
# 'buyer_supplier_3m_dr', 
'buyer_supplier_3m_term', 
'buyer_supplier_3m_late_days', 
# 'buyer_supplier_3m_isLate', 
'buyer_supplier_6m_inv_cnt', 
'buyer_supplier_6m_ave_fin_amt', 
'buyer_supplier_6m_std_fin_amt', 
# 'buyer_supplier_6m_dr', 
'buyer_supplier_6m_term', 
'buyer_supplier_6m_late_days', 
# 'buyer_supplier_6m_isLate', 
'client_yrs', 
'client_past_inv_cnt', 
'client_annual_inv_cnt', 
'client_ave_fin_amt', 
'client_ave_annual_fin_amt', 
'client_std_fin_amt', 
'client_late_days', 
# 'client_isLate', 
'client_dr', 
'client_term', 
'client_cty', 
'client_prevyr_inv_cnt', 
'client_prevyr_ave_fin_amt', 
'client_prevyr_std_fin_amt', 
'client_prevyr_late_days', 
# 'client_prevyr_isLate', 
# 'client_prevyr_dr', 
'client_prevyr_term', 
'client_season_x', 
'client_6m_inv_cnt', 
'client_6m_ave_fin_amt', 
'client_6m_std_fin_amt', 
# 'client_6m_dr', 
'client_6m_term', 
'client_6m_late_days', 
# 'client_6m_isLate', 
'client_3m_inv_cnt', 
'client_3m_ave_fin_amt', 
'client_3m_std_fin_amt', 
# 'client_3m_dr', 
'client_3m_term', 
'client_3m_late_days', 
# 'client_3m_isLate', 
# 'first_client_invoice', 
# 'client_id', 
'industry_code'                    ]]

uncast_cols = feature_set.columns

In [17]:
feature_set1=pd.get_dummies(feature_set)
feature_set1.rename(columns = {x:x.replace("__","_") for x in feature_set1 if "__" in x}, inplace = True)
feature_set1.columns

Index([u'tenor', u'grace_period', u'fin_tenor', u'doc_usd_amt',
       u'due_date_dow', u'maturity_date_dow', u'buyer_centric_flag',
       u'ccy_wk_vol', u'ccy_3m_vol', u'ccy_6m_vol',
       ...
       u'CG_5B', u'CG_6A', u'CG_6B', u'CG_7A', u'CG_7B', u'CG_8A', u'CG_8B',
       u'CG_9A', u'CG_9B', u'CG_NO_CG'],
      dtype='object', length=168)

In [18]:
feature_cols1 = [col for col in feature_set1.columns]
len(feature_cols1)

168

In [19]:
cast_frame = pd.read_csv(data_dir+'cast_columns_template.csv')
cast_frame.tail()

,tenor,grace_period,fin_tenor,doc_usd_amt,due_date_dow,maturity_date_dow,buyer_centric_flag,ccy_wk_vol,ccy_3m_vol,ccy_6m_vol,ccy_yr_vol,ccy_tenor_vol,ccy_fintenor_vol,ccy_wk_vector,ccy_3m_vector,ccy_6m_vector,ccy_yr_vector,ccy_tenor_vector,ccy_fintenor_vector,RICI_index,buyer_supplier_prevyr_inv_cnt,buyer_supplier_prevyr_ave_fin_amt,buyer_supplier_prevyr_std_fin_amt,buyer_supplier_prevyr_late_days,buyer_supplier_prevyr_dr,buyer_supplier_prevyr_term,buyer_supplier_season_x,buyer_supplier_yrs,buyer_supplier_past_inv_cnt,buyer_supplier_annual_inv_cnt,buyer_supplier_ave_fin_amt,buyer_supplier_ave_annual_fin_amt,buyer_supplier_std_fin_amt,buyer_supplier_late_days,buyer_supplier_term,buyer_supplier_cty,buyer_supplier_3m_inv_cnt,buyer_supplier_3m_ave_fin_amt,buyer_supplier_3m_std_fin_amt,buyer_supplier_3m_term,buyer_supplier_3m_late_days,buyer_supplier_6m_inv_cnt,buyer_supplier_6m_ave_fin_amt,buyer_supplier_6m_std_fin_amt,buyer_supplier_6m_term,buyer_supplier_6m_late_days,client_yrs,client_past_inv_cnt,client_annual_inv_cnt,client_ave_fin_amt,client_ave_annual_fin_amt,client_std_fin_amt,client_late_days,client_dr,client_term,client_cty,client_prevyr_inv_cnt,client_prevyr_ave_fin_amt,client_prevyr_std_fin_amt,client_prevyr_late_days,client_prevyr_term,client_season_x,client_6m_inv_cnt,client_6m_ave_fin_amt,client_6m_std_fin_amt,client_6m_term,client_6m_late_days,client_3m_inv_cnt,client_3m_ave_fin_amt,client_3m_std_fin_amt,client_3m_term,client_3m_late_days,industry_code,cty_code_AE,cty_code_BD,cty_code_CN,cty_code_DF,cty_code_GB,cty_code_HK,cty_code_ID,cty_code_IN,cty_code_JO,cty_code_JP,cty_code_KE,cty_code_LK,cty_code_MO,cty_code_MY,cty_code_NG,cty_code_PH,cty_code_PK,cty_code_QA,cty_code_SG,cty_code_SP,cty_code_TH,cty_code_TW,cty_code_TZ,cty_code_UG,cty_code_US,cty_code_VN,cty_code_ZA,doc_ccy_code_AED,doc_ccy_code_AUD,doc_ccy_code_CAD,doc_ccy_code_CHF,doc_ccy_code_CNY,doc_ccy_code_EUR,doc_ccy_code_GBP,doc_ccy_code_HKD,doc_ccy_code_IDR,doc_ccy_code_INR,doc_ccy_code_JPY,doc_ccy_code_KES,doc_ccy_code_LKR,doc_ccy_code_MYR,doc_ccy_code_NGN,doc_ccy_code_NZD,doc_ccy_code_PHP,doc_ccy_code_PKR,doc_ccy_code_SGD,doc_ccy_code_THB,doc_ccy_code_TZS,doc_ccy_code_USD,doc_ccy_code_VND,doc_ccy_code_ZAR,fin_ccy_code_AED,fin_ccy_code_AUD,fin_ccy_code_CAD,fin_ccy_code_CHF,fin_ccy_code_CNY,fin_ccy_code_EUR,fin_ccy_code_GBP,fin_ccy_code_HKD,fin_ccy_code_IDR,fin_ccy_code_INR,fin_ccy_code_JPY,fin_ccy_code_KES,fin_ccy_code_LKR,fin_ccy_code_MYR,fin_ccy_code_NGN,fin_ccy_code_NZD,fin_ccy_code_PHP,fin_ccy_code_PKR,fin_ccy_code_SGD,fin_ccy_code_THB,fin_ccy_code_TZS,fin_ccy_code_USD,fin_ccy_code_VND,fin_ccy_code_ZAR,product_variant_code_BATB01,product_variant_code_ECIF01,product_variant_code_ECPR01,product_variant_code_IFNB01,product_variant_code_IFNB02,product_variant_code_IFNB03,product_variant_code_IFNB04,product_variant_code_IFNB05,product_variant_code_IFNS01,product_variant_code_IFNS02,product_variant_code_IFNS03,product_variant_code_IFNS05,product_variant_code_LBDS01,product_variant_code_LBDS03,product_variant_code_LRRP02,product_variant_code_PSHP02,product_variant_code_RSRV01,product_variant_code_RSRV02,product_variant_code_RSRV03,product_variant_code_RSRV05,product_variant_code_RSRV06,product_variant_code_RSRV07,product_variant_code_RSRV08,product_variant_code_TRDS01,product_BATB,product_ECIF,product_ECPR,product_IFNB,product_IFNS,product_LBDS,product_LRRP,product_PSHP,product_RSRV,product_TRDS,CG_10A,CG_10B,CG_11A,CG_11B,CG_11C,CG_12B,CG_12C,CG_13,CG_14,CG_1A,CG_1B,CG_2A,CG_2B,CG_3A,CG_3B,CG_4A,CG_4B,CG_5A,CG_5B,CG_6A,CG_6B,CG_7A,CG_7B,CG_8A,CG_8B,CG_9A,CG_9B,CG_NO_CG


In [ ]:
[x for x in feature_cols1 if x not in cast_frame.columns]

In [20]:
for col in cast_frame.columns:
    if col not in feature_set1.columns:
        feature_set1[col] = [0 for i in range(len(feature_set1))]
        
[x for x in cast_frame.columns if x not in feature_set1.columns], len(feature_set1.columns)

([], 210)

In [21]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

from sklearn.externals import joblib


In [22]:
# scale, joblib scaler
scaler = joblib.load(data_dir+'minmaxscaler_180223.pkl')
feature_set1=scaler.fit_transform(feature_set1)
type(feature_set1)


/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


numpy.ndarray

In [29]:
final =data
final.index = range(len(final))
final['rownum'] = final.index

/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
from sklearn.metrics import confusion_matrix as cm

In [26]:
rf = joblib.load(data_dir+'rf_ld_180223.joblib')

/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [27]:
rf.n_features_

210

In [30]:
### round,3
ld_pred= rf.predict(feature_set1)
ld_rf_prob = rf.predict_proba(feature_set1)

ld_rf_preds = [[final.doc_id[i], final.late_days_bucket[i], ld_pred[i], ld_rf_prob[i][0], 
                    ld_rf_prob[i][1], ld_rf_prob[i][2],ld_rf_prob[i][3]] for i in range(len(final))]

ld_rf_preds = pd.DataFrame(ld_rf_preds, columns = ['doc_id', 'late_days_bucket', 'isLate_predict', 'late_prob_0', 'late_prob_1', 
                                                   'late_prob_2', 'late_prob_3'])

ld_rf_preds.to_csv(data_dir+'ld_rf_preds.csv')

ld_rf_preds.head()

,doc_id,late_days_bucket,isLate_predict,late_prob_0,late_prob_1,late_prob_2,late_prob_3
0,RFHLM9WX4V-89034,0,0,0.5,0.3,0.1,0.1
1,V5HKSF85PW-36933,0,0,1.0,0.0,0.0,0.0
2,52HNXCS9NQ-6901,3,1,0.1,0.5,0.2,0.2
3,52HNXCS9NQ-6902,3,1,0.1,0.5,0.2,0.2
4,52HNXCS9NQ-6968,3,3,0.1,0.1,0.0,0.8


In [31]:
print(cm(data.late_days_bucket, ld_pred))
print(cm(data.late_days_bucket, ld_pred)*1./(len(data)*1.))
print(accuracy_score(data.late_days_bucket, ld_pred))

[[93767 29033   145   242]
 [70826 92486   800  4186]
 [  968   879    24   358]
 [ 1711    10     0  1185]]
[[  3.16118266e-01   9.78794417e-02   4.88840941e-04   8.15858674e-04]
 [  2.38776886e-01   3.11799609e-01   2.69705347e-03   1.41123323e-02]
 [  3.26343470e-03   2.96338750e-03   8.09116041e-05   1.20693143e-03]
 [  5.76832311e-03   3.37131684e-05   0.00000000e+00   3.99501045e-03]]
0.631993796777


### DR

In [32]:

print("RF")

rf = joblib.load(data_dir+'rf_dr_180223.joblib')

dr_pred= rf.predict(feature_set1)
dr_rf_prob = rf.predict_proba(feature_set1)

dr_rf_preds = [[final.doc_id[i], final.late_days_bucket[i], dr_pred[i], dr_rf_prob[i][0], 
                    dr_rf_prob[i][1], dr_rf_prob[i][2]] for i in range(len(final))]

dr_rf_preds = pd.DataFrame(dr_rf_preds, columns = ['doc_id', 'dr_bucket', 'dr_predict', 'dr_prob_0', 'dr_prob_1', 
                                                   'dr_prob_2'])

dr_rf_preds.to_csv(data_dir+'dr_rf_preds.csv')

print(cm(data.dr_bucket, dr_pred))
dr_rf_preds.head()

RF
[[206688  23137  15228]
 [ 20237  22055   4282]
 [  3505    453   1035]]


,doc_id,dr_bucket,dr_predict,dr_prob_0,dr_prob_1,dr_prob_2
0,RFHLM9WX4V-89034,0,1,0.3,0.4,0.3
1,V5HKSF85PW-36933,0,1,0.2,0.5,0.3
2,52HNXCS9NQ-6901,3,1,0.1,0.7,0.2
3,52HNXCS9NQ-6902,3,1,0.2,0.7,0.1
4,52HNXCS9NQ-6968,3,2,0.4,0.1,0.5


In [33]:
[x*1./len(data)*1. for x in cm(data.dr_bucket, dr_pred)]

[array([ 0.69681073,  0.07800216,  0.05133841]),
 array([ 0.06822534,  0.07435439,  0.01443598]),
 array([ 0.01181647,  0.00152721,  0.00348931])]

### 

In [34]:
cm(data.dr_bucket, dr_pred)*1./(len(data)*1.)

array([[ 0.69681073,  0.07800216,  0.05133841],
       [ 0.06822534,  0.07435439,  0.01443598],
       [ 0.01181647,  0.00152721,  0.00348931]])

In [35]:
accuracy_score(data.dr_bucket, dr_pred)

0.77465444002427353

In [36]:
!zip /home/flowcast/data/python/data_validate_180315/validate_preds.zip /home/flowcast/data/python/data_validate_180315/dr_rf_preds.csv /home/flowcast/data/python/data_validate_180315/ld_rf_preds.csv

  adding: home/flowcast/data/python/data_validate_180315/dr_rf_preds.csv (deflated 86%)
  adding: home/flowcast/data/python/data_validate_180315/ld_rf_preds.csv (deflated 87%)


In [37]:
len(data)

296620

In [38]:
data.ix[:196620]

/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,cty_code,doc_id,supplier_id,buyer_id,doc_type_code,deal_id,fin_flag,rec_id,doc_date,doc_ccy_code,doc_ccy_amt,tenor,tenor_start_date,due_date,grace_period,maturity_date,fin_tenor_start_date,fin_tenor,to_seq_no,from_seq_no,pymt_ccy_code,pymt_alloc_amt,fin_ccy_code,fin_ccy_cp_alloc_amt,fin_ccy_ncp_alloc_amt,fin_ccy_ncp_os_amt,fin_ccy_cp_os_amt,fin_ccy_amt,os_ccy_amt,fin_date,sale_ledger_flag,pool_based_flag,step_code,product_variant_code,rel_seq_no,pymt_date,settled_flag,late_days,finance_days,finance_due_days,isLate,late_days_bucket,fin_xrate,pymt_xrate,doc_xrate,doc_usd_amt,fin_usd_amt,os_usd_amt,pymt_usd_amt,dr,dr_bucket,product_,client_id,le_id,lid_isic_cd_val,industry_code,CG,buyer_supplier_yrs,buyer_supplier_past_inv_cnt,buyer_supplier_annual_inv_cnt,buyer_supplier_ave_fin_amt,buyer_supplier_ave_annual_fin_amt,buyer_supplier_std_fin_amt,buyer_supplier_late_days,buyer_supplier_isLate,buyer_supplier_dr,buyer_supplier_term,buyer_supplier_cty,buyer_supplier_prevyr_inv_cnt,buyer_supplier_prevyr_ave_fin_amt,buyer_supplier_prevyr_std_fin_amt,buyer_supplier_prevyr_late_days,buyer_supplier_prevyr_isLate,buyer_supplier_prevyr_dr,buyer_supplier_prevyr_term,buyer_supplier_season_x,buyer_supplier_6m_inv_cnt,buyer_supplier_6m_ave_fin_amt,buyer_supplier_6m_std_fin_amt,buyer_supplier_6m_dr,buyer_supplier_6m_term,buyer_supplier_6m_late_days,buyer_supplier_6m_isLate,buyer_supplier_3m_inv_cnt,buyer_supplier_3m_ave_fin_amt,buyer_supplier_3m_std_fin_amt,buyer_supplier_3m_dr,buyer_supplier_3m_term,buyer_supplier_3m_late_days,buyer_supplier_3m_isLate,client_yrs,client_past_inv_cnt,client_annual_inv_cnt,client_ave_fin_amt,client_ave_annual_fin_amt,client_std_fin_amt,client_late_days,client_isLate,client_dr,client_term,client_cty,client_3m_inv_cnt,client_3m_ave_fin_amt,client_3m_std_fin_amt,client_3m_dr,client_3m_term,client_3m_late_days,client_3m_isLate,client_season_x,client_6m_inv_cnt,client_6m_ave_fin_amt,client_6m_std_fin_amt,client_6m_dr,client_6m_term,client_6m_late_days,client_6m_isLate,client_prevyr_inv_cnt,client_prevyr_ave_fin_amt,client_prevyr_std_fin_amt,client_prevyr_late_days,client_prevyr_isLate,client_prevyr_dr,client_prevyr_term,first_client_invoice,RICI_index,due_date_dow,maturity_date_dow,buyer_centric_flag,ccy_wk_vector,ccy_3m_vector,ccy_6m_vector,ccy_yr_vector,ccy_tenor_vector,ccy_fintenor_vector,ccy_wk_vol,ccy_3m_vol,ccy_6m_vol,ccy_yr_vol,ccy_tenor_vol,ccy_fintenor_vol,rownum
0,AE,RFHLM9WX4V-89034,100369293,100244505.0,INV,TF123BF562,Y,6728531788,2017-03-24,USD,113268.50,45,2017-03-24 00:00:00,2017-05-08,0,2017-05-08,2017-03-25 00:00:00,44.0,37.0,1,USD,113268.50,USD,0.00,113268.50,0.0,0.0,113268.50,0.0,2017-03-25,Y,N,SETT,IFNB02,37.0,2017-05-04,Y,0,40,44,0,0,1.000000,1.000000,1.000000,113268.500000,113268.500000,0.0,113268.500000,0.000000,0,IFNB,100244505,11545630.0,6116.0,61,5A,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2229.32,0,0,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0
1,AE,V5HKSF85PW-36933,100369293,100244505.0,INV,TF123BF144,Y,783293897,2017-03-10,USD,106717.15,45,2017-03-10 00:00:00,2017-04-24,0,2017-04-24,2017-03-11 00:00:00,44.0,29.0,1,USD,106717.15,USD,0.00,106717.15,0.0,0.0,106717.15,0.0,2017-03-11,Y,N,SETT,IFNB02,29.0,2017-04-09,Y,0,29,44,0,0,1.000000,1.000000,1.000000,106717.150000,106717.150000,0.0,106717.150000,0.000000,0,IFNB,100244505,11545630.0,6116.0,61,5A,1.0,7.0,7.0,107842.878571,7.549002e+05,43236.173824,0.000000,0.000000,0.000000,44.428571,1.0,7.0,107842.878571,43236.173824,0.0000

In [40]:
data.ix[196620:196622]

,cty_code,doc_id,supplier_id,buyer_id,doc_type_code,deal_id,fin_flag,rec_id,doc_date,doc_ccy_code,doc_ccy_amt,tenor,tenor_start_date,due_date,grace_period,maturity_date,fin_tenor_start_date,fin_tenor,to_seq_no,from_seq_no,pymt_ccy_code,pymt_alloc_amt,fin_ccy_code,fin_ccy_cp_alloc_amt,fin_ccy_ncp_alloc_amt,fin_ccy_ncp_os_amt,fin_ccy_cp_os_amt,fin_ccy_amt,os_ccy_amt,fin_date,sale_ledger_flag,pool_based_flag,step_code,product_variant_code,rel_seq_no,pymt_date,settled_flag,late_days,finance_days,finance_due_days,isLate,late_days_bucket,fin_xrate,pymt_xrate,doc_xrate,doc_usd_amt,fin_usd_amt,os_usd_amt,pymt_usd_amt,dr,dr_bucket,product_,client_id,le_id,lid_isic_cd_val,industry_code,CG,buyer_supplier_yrs,buyer_supplier_past_inv_cnt,buyer_supplier_annual_inv_cnt,buyer_supplier_ave_fin_amt,buyer_supplier_ave_annual_fin_amt,buyer_supplier_std_fin_amt,buyer_supplier_late_days,buyer_supplier_isLate,buyer_supplier_dr,buyer_supplier_term,buyer_supplier_cty,buyer_supplier_prevyr_inv_cnt,buyer_supplier_prevyr_ave_fin_amt,buyer_supplier_prevyr_std_fin_amt,buyer_supplier_prevyr_late_days,buyer_supplier_prevyr_isLate,buyer_supplier_prevyr_dr,buyer_supplier_prevyr_term,buyer_supplier_season_x,buyer_supplier_6m_inv_cnt,buyer_supplier_6m_ave_fin_amt,buyer_supplier_6m_std_fin_amt,buyer_supplier_6m_dr,buyer_supplier_6m_term,buyer_supplier_6m_late_days,buyer_supplier_6m_isLate,buyer_supplier_3m_inv_cnt,buyer_supplier_3m_ave_fin_amt,buyer_supplier_3m_std_fin_amt,buyer_supplier_3m_dr,buyer_supplier_3m_term,buyer_supplier_3m_late_days,buyer_supplier_3m_isLate,client_yrs,client_past_inv_cnt,client_annual_inv_cnt,client_ave_fin_amt,client_ave_annual_fin_amt,client_std_fin_amt,client_late_days,client_isLate,client_dr,client_term,client_cty,client_3m_inv_cnt,client_3m_ave_fin_amt,client_3m_std_fin_amt,client_3m_dr,client_3m_term,client_3m_late_days,client_3m_isLate,client_season_x,client_6m_inv_cnt,client_6m_ave_fin_amt,client_6m_std_fin_amt,client_6m_dr,client_6m_term,client_6m_late_days,client_6m_isLate,client_prevyr_inv_cnt,client_prevyr_ave_fin_amt,client_prevyr_std_fin_amt,client_prevyr_late_days,client_prevyr_isLate,client_prevyr_dr,client_prevyr_term,first_client_invoice,RICI_index,due_date_dow,maturity_date_dow,buyer_centric_flag,ccy_wk_vector,ccy_3m_vector,ccy_6m_vector,ccy_yr_vector,ccy_tenor_vector,ccy_fintenor_vector,ccy_wk_vol,ccy_3m_vol,ccy_6m_vol,ccy_yr_vol,ccy_tenor_vol,ccy_fintenor_vol,rownum
196620,US,B7HWBN5BKJ-20387,100306891,100306803.0,INV,TF777R0069,Y,7035190704,2017-05-20,USD,266.78,90,2017-05-20 00:00:00,2017-08-18,0,2017-08-24,2017-05-20 00:00:00,96.0,6429.0,5810,USD,266.78,USD,266.78,0.0,0.0,0.0,266.78,0.0,2017-05-26,Y,N,SETT,RSRV01,6429.0,2017-08-24,Y,6,90,84,1,1,1.0,1.0,1.0,266.78,266.78,0.0,266.78,0.0,0,RSRV,100306891,13374205.0,8342.0,83,6B,1.0,90979.0,90979.0,2352.43174,2.140219e+08,126383.758133,2.888908,0.866266,0.0,90.732158,1.0,54916.0,1245.012073,11968.172246,2.473432,0.824222,0.0,92.473432,0.214279,42699.0,1325.369748,13001.079805,0.0,92.469449,2.469449,0.824305,34725.0,1428.474992,13191.014696,0.0,92.41365,2.41365,0.817912,1.0,90979.0,90979.0,2352.43174,2.140219e+08,126383.758133,2.888908,0.866266,0.0,90.732158,1.0,34725.0,1428.474992,13191.014696,0.0,92.41365,2.41365,0.817912,0.214279,42699.0,1325.369748,13001.079805,0.0,92.469449,2.469449,0.824305,54916.0,1245.012073,11968.172246,2.473432,0.824222,0.0,92.473432,0,0.0,4,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196620
196621,US,B7HWBN5BKJ-20388,100306891,100306803.0,INV,TF777R0069,Y,7035190702,2017-05-20,USD,373.66,90,2017-05-20 00:00:00,2017-08-18,0,2017-08-24,2017-05-20 00:00:00,96.0,6429.0,5810,USD,373.66,USD,373.66,0.0,0.0,0.0,373.66,0.0,2017-05-26,Y,N,SETT,RSRV01,6429.0,2017-08-24,Y,6,90,84,1,1,1.0,1.0,1.0,373.66,373.66,0.0,373.66,0.0,0,RSRV,100306891,13374205.0,8342.0,83,6B,1.0,90979.0,90979.0,2352.43174,2.140219e+08,126383.758133,2.888908,0.866266,0.0,90.732158,1.0,54916.0,1245.012073,11968.172246,2.473432,0.824222,0.0,92.473432,0.300126,42699.0,1325.3

In [41]:
data1 = data.ix[:196620]
data2 = data.ix[196621:]
len(data1), len(data2), len(data1)+len(data2), len(data)

(196621, 99999, 296620, 296620)

In [42]:
data1.to_csv(data_dir+'data_for_model1.csv', index= False)
data2.to_csv(data_dir+'data_for_model2.csv', index= False)

In [43]:
!zip /home/flowcast/data/python/data_validate_180315/data_for_model1.zip /home/flowcast/data/python/data_validate_180315/data_for_model1.csv

  adding: home/flowcast/data/python/data_validate_180315/data_for_model1.csv (deflated 91%)


In [44]:
!zip /home/flowcast/data/python/data_validate_180315/data_for_model2.zip /home/flowcast/data/python/data_validate_180315/data_for_model2.csv

  adding: home/flowcast/data/python/data_validate_180315/data_for_model2.csv (deflated 94%)


In [45]:
scaled_df = pd.DataFrame(data = feature_set1, columns=cast_frame.columns)
scaled_df.tail()

,tenor,grace_period,fin_tenor,doc_usd_amt,due_date_dow,maturity_date_dow,buyer_centric_flag,ccy_wk_vol,ccy_3m_vol,ccy_6m_vol,ccy_yr_vol,ccy_tenor_vol,ccy_fintenor_vol,ccy_wk_vector,ccy_3m_vector,ccy_6m_vector,ccy_yr_vector,ccy_tenor_vector,ccy_fintenor_vector,RICI_index,buyer_supplier_prevyr_inv_cnt,buyer_supplier_prevyr_ave_fin_amt,buyer_supplier_prevyr_std_fin_amt,buyer_supplier_prevyr_late_days,buyer_supplier_prevyr_dr,buyer_supplier_prevyr_term,buyer_supplier_season_x,buyer_supplier_yrs,buyer_supplier_past_inv_cnt,buyer_supplier_annual_inv_cnt,buyer_supplier_ave_fin_amt,buyer_supplier_ave_annual_fin_amt,buyer_supplier_std_fin_amt,buyer_supplier_late_days,buyer_supplier_term,buyer_supplier_cty,buyer_supplier_3m_inv_cnt,buyer_supplier_3m_ave_fin_amt,buyer_supplier_3m_std_fin_amt,buyer_supplier_3m_term,buyer_supplier_3m_late_days,buyer_supplier_6m_inv_cnt,buyer_supplier_6m_ave_fin_amt,buyer_supplier_6m_std_fin_amt,buyer_supplier_6m_term,buyer_supplier_6m_late_days,client_yrs,client_past_inv_cnt,client_annual_inv_cnt,client_ave_fin_amt,client_ave_annual_fin_amt,client_std_fin_amt,client_late_days,client_dr,client_term,client_cty,client_prevyr_inv_cnt,client_prevyr_ave_fin_amt,client_prevyr_std_fin_amt,client_prevyr_late_days,client_prevyr_term,client_season_x,client_6m_inv_cnt,client_6m_ave_fin_amt,client_6m_std_fin_amt,client_6m_term,client_6m_late_days,client_3m_inv_cnt,client_3m_ave_fin_amt,client_3m_std_fin_amt,client_3m_term,client_3m_late_days,industry_code,cty_code_AE,cty_code_BD,cty_code_CN,cty_code_DF,cty_code_GB,cty_code_HK,cty_code_ID,cty_code_IN,cty_code_JO,cty_code_JP,cty_code_KE,cty_code_LK,cty_code_MO,cty_code_MY,cty_code_NG,cty_code_PH,cty_code_PK,cty_code_QA,cty_code_SG,cty_code_SP,cty_code_TH,cty_code_TW,cty_code_TZ,cty_code_UG,cty_code_US,cty_code_VN,cty_code_ZA,doc_ccy_code_AED,doc_ccy_code_AUD,doc_ccy_code_CAD,doc_ccy_code_CHF,doc_ccy_code_CNY,doc_ccy_code_EUR,doc_ccy_code_GBP,doc_ccy_code_HKD,doc_ccy_code_IDR,doc_ccy_code_INR,doc_ccy_code_JPY,doc_ccy_code_KES,doc_ccy_code_LKR,doc_ccy_code_MYR,doc_ccy_code_NGN,doc_ccy_code_NZD,doc_ccy_code_PHP,doc_ccy_code_PKR,doc_ccy_code_SGD,doc_ccy_code_THB,doc_ccy_code_TZS,doc_ccy_code_USD,doc_ccy_code_VND,doc_ccy_code_ZAR,fin_ccy_code_AED,fin_ccy_code_AUD,fin_ccy_code_CAD,fin_ccy_code_CHF,fin_ccy_code_CNY,fin_ccy_code_EUR,fin_ccy_code_GBP,fin_ccy_code_HKD,fin_ccy_code_IDR,fin_ccy_code_INR,fin_ccy_code_JPY,fin_ccy_code_KES,fin_ccy_code_LKR,fin_ccy_code_MYR,fin_ccy_code_NGN,fin_ccy_code_NZD,fin_ccy_code_PHP,fin_ccy_code_PKR,fin_ccy_code_SGD,fin_ccy_code_THB,fin_ccy_code_TZS,fin_ccy_code_USD,fin_ccy_code_VND,fin_ccy_code_ZAR,product_variant_code_BATB01,product_variant_code_ECIF01,product_variant_code_ECPR01,product_variant_code_IFNB01,product_variant_code_IFNB02,product_variant_code_IFNB03,product_variant_code_IFNB04,product_variant_code_IFNB05,product_variant_code_IFNS01,product_variant_code_IFNS02,product_variant_code_IFNS03,product_variant_code_IFNS05,product_variant_code_LBDS01,product_variant_code_LBDS03,product_variant_code_LRRP02,product_variant_code_PSHP02,product_variant_code_RSRV01,product_variant_code_RSRV02,product_variant_code_RSRV03,product_variant_code_RSRV05,product_variant_code_RSRV06,product_variant_code_RSRV07,product_variant_code_RSRV08,product_variant_code_TRDS01,product_BATB,product_ECIF,product_ECPR,product_IFNB,product_IFNS,product_LBDS,product_LRRP,product_PSHP,product_RSRV,product_TRDS,CG_10A,CG_10B,CG_11A,CG_11B,CG_11C,CG_12B,CG_12C,CG_13,CG_14,CG_1A,CG_1B,CG_2A,CG_2B,CG_3A,CG_3B,CG_4A,CG_4B,CG_5A,CG_5B,CG_6A,CG_6B,CG_7A,CG_7B,CG_8A,CG_8B,CG_9A,CG_9B,CG_NO_CG
296615,0.097222,0.0,0.180556,0.000166,0.166667,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.085615,0.003933,0.003609,0.003328,0.00313,0.00313,0.914044,0.000429,0.000262,0.000157,0.172798,0.137867,0.175775,6.500188e-07,1.0,0.000292,0.000292,0.000146,0.000123,0.000123,0.172798,0.098826,0.5,0.000774,0.000140,0.000192,0.181759,0.185455,0.000469,0.000142,0.000184,0.181818,0.185491,1.0,0.006539,0.006539,0.

In [46]:
scaled_df.ix[:150000].to_csv(data_dir+'scaled_data1.csv', index= False)
scaled_df.ix[150001:].to_csv(data_dir+'scaled_data2.csv', index= False)

/data/flowcast/data/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [47]:
!zip /home/flowcast/data/python/data_validate_180315/scaled_data1.zip /home/flowcast/data/python/data_validate_180315/scaled_data1.csv

  adding: home/flowcast/data/python/data_validate_180315/scaled_data1.csv (deflated 94%)


In [48]:
!zip /home/flowcast/data/python/data_validate_180315/scaled_data2.zip /home/flowcast/data/python/data_validate_180315/scaled_data2.csv

  adding: home/flowcast/data/python/data_validate_180315/scaled_data2.csv (deflated 96%)
